<a href="https://colab.research.google.com/github/dhanushnayak/BERT/blob/main/Kantri_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Byte to Byte level pair encoding tokenizer, separation token "</ s>"

In [1]:
!curl -L https://raw.githubusercontent.com/PacktPublishing/Transformers-for-Natural-Language-Processing/main/Chapter03/kant.txt --output "kant.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.7M  100 10.7M    0     0  7677k      0  0:00:01  0:00:01 --:--:-- 7672k


In [2]:
!pip uninstall -y tensorflow

Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220527125636


In [3]:
!pip install git+https://github.com/huggingface/transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-g5tvgoo6
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-g5tvgoo6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 36.6 MB/s 
     |████████████████████████████████| 101 kB 14.7 MB/s 
     |████████████████████████████████| 6.6 MB 53.0 MB/s 
  Created wheel for transformers: filename=transformers-4.21.0.dev0-py3-none-any.whl size=4496947 sha256=a0c994ed4c4891940a64143db36f76bb071ba1adb73c6a20a619320cc45898b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-81dyuo9m/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Unins

In [4]:
!pip list | grep -E "transformers|tokenizers"

tokenizers                    0.12.1
transformers                  4.21.0.dev0


In [5]:
%%time 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
paths =[str(x) for x in  Path('.').glob("**/*.txt")]

CPU times: user 10.2 ms, sys: 3.07 ms, total: 13.3 ms
Wall time: 16.3 ms


Tokenizer_creation

In [6]:
tokenizer = ByteLevelBPETokenizer()

Training _token 

In [7]:
tokenizer.train(files=paths,vocab_size=52000,min_frequency=2,special_tokens=[
     "<s>","<pad>","</s>","<unk>","<mask>"                                                                 
])

saving token

In [8]:
import os
token_dir = '/content/KantaiBERT'
if not os.path.exists(token_dir):
  os.makedirs(token_dir)
tokenizer.save_model("KantaiBERT")

['KantaiBERT/vocab.json', 'KantaiBERT/merges.txt']

In [9]:
with open("kant.txt",'r') as f:
  for i,j in enumerate(f):
    if i==10:
      break
    print(j)



The Project Gutenberg EBook of The Critique of Pure Reason, by Immanuel Kant



This eBook is for the use of anyone anywhere at no cost and with

almost no restrictions whatsoever.  You may copy it, give it away or

re-use it under the terms of the Project Gutenberg License included

with this eBook or online at www.gutenberg.net





Title: The Critique of Pure Reason



In [10]:
with open("KantaiBERT/merges.txt",'r') as f:
  for i,j in enumerate(f):
    if i==10:
      break
    print(j)

#version: 0.2 - Trained by `huggingface/tokenizers`

Ġ t

h e

Ġ a

o n

i n

Ġ o

Ġt he

r e

i t



reloading the checking token

In [11]:
from tokenizers.processors import BertProcessing

In [12]:
tokenizer = ByteLevelBPETokenizer(
    vocab = "./KantaiBERT/vocab.json",
    merges = "./KantaiBERT/merges.txt"
)

In [13]:
tokenizer.encode("Hello data k++")

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

post_processor will add start and end

In [14]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>",tokenizer.token_to_id("</s>")),
    ("<s>",tokenizer.token_to_id("<s>"))
)

In [15]:
tokenizer.enable_truncation(max_length=512)

In [16]:
tokenizer.encode("hemllop sd fjf").tokens

['<s>', 'he', 'm', 'll', 'op', 'Ġs', 'd', 'Ġf', 'j', 'f', '</s>']

In [17]:
!nvidia-smi

Tue Jun 28 09:26:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
import torch
torch.cuda.is_available()

True

<h1> RobertaConfig </h1>

In [19]:
from transformers import RobertaConfig
config = RobertaConfig(
    vocab_size=52000,
    max_position_embeddings=514,
    num_attention_head=12,
    num_hidden_layers = 6,
    type_vocab_size=1
)

loading kantaiBert tokenizer in transformers

In [22]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("./KantaiBERT",max_length=512)

Model building, mask language

In [23]:
from transformers import RobertaForMaskedLM

In [24]:
model = RobertaForMaskedLM(config=config)

In [25]:
print(model)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [26]:
print(model.num_parameters())

83504416


In [29]:
LP = list(model.parameters())
len(LP)

106

DataLoader

In [37]:
%%time
# loading dataset line by line by limiting the length os block_size=128
from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(tokenizer,"./kant.txt",block_size=128)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 25.5 s, sys: 233 ms, total: 25.7 s
Wall time: 25.7 s


Batch Maker with mlm activation

In [38]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=True,mlm_probability=0.15)

In [40]:
data_collator

DataCollatorForLanguageModeling(tokenizer=PreTrainedTokenizer(name_or_path='./KantaiBERT', vocab_size=19296, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}), mlm=True, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=Fa

Trainer

In [41]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./KantaiBERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=1000,
    save_total_limit=2
)

In [44]:
trainer = Trainer(model=model,args=training_args,data_collator=data_collator,train_dataset=dataset)

In [45]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 170964
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2672


Step,Training Loss
500,6.593000
1000,5.729000
1500,5.255500
2000,4.994900
2500,4.847700


Saving model checkpoint to ./KantaiBERT/checkpoint-1000
Configuration saved in ./KantaiBERT/checkpoint-1000/config.json
Model weights saved in ./KantaiBERT/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./KantaiBERT/checkpoint-2000
Configuration saved in ./KantaiBERT/checkpoint-2000/config.json
Model weights saved in ./KantaiBERT/checkpoint-2000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2672, training_loss=5.440053517233112, metrics={'train_runtime': 579.4556, 'train_samples_per_second': 295.042, 'train_steps_per_second': 4.611, 'total_flos': 873620128952064.0, 'train_loss': 5.440053517233112, 'epoch': 1.0})

Save Model

In [46]:
trainer.save_model("./KantaiBERT")

Saving model checkpoint to ./KantaiBERT
Configuration saved in ./KantaiBERT/config.json
Model weights saved in ./KantaiBERT/pytorch_model.bin


**Pipeline**

In [47]:
from transformers import pipeline

fill_mask = pipeline("fill-mask",model='./KantaiBERT',tokenizer="./KantaiBERT")

loading configuration file ./KantaiBERT/config.json
Model config RobertaConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_head": 12,
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.21.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file ./KantaiBERT/config.json
Model config RobertaConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_tok

In [49]:
fill_mask("my name is dhanush and <mask> who work on data set")

[{'score': 0.05704280361533165,
  'sequence': 'my name is dhanush and it who work on data set',
  'token': 306,
  'token_str': ' it'},
 {'score': 0.04404142498970032,
  'sequence': 'my name is dhanush and to who work on data set',
  'token': 288,
  'token_str': ' to'},
 {'score': 0.04077835753560066,
  'sequence': 'my name is dhanush and the who work on data set',
  'token': 267,
  'token_str': ' the'},
 {'score': 0.033392488956451416,
  'sequence': 'my name is dhanush and, who work on data set',
  'token': 16,
  'token_str': ','},
 {'score': 0.02472991682589054,
  'sequence': 'my name is dhanush and not who work on data set',
  'token': 339,
  'token_str': ' not'}]

In [53]:
fill_mask("movie release is leaked by <mask> tamil ")

[{'score': 0.1469995081424713,
  'sequence': 'movie release is leaked by the tamil ',
  'token': 267,
  'token_str': ' the'},
 {'score': 0.047932498157024384,
  'sequence': 'movie release is leaked by this tamil ',
  'token': 342,
  'token_str': ' this'},
 {'score': 0.04595071077346802,
  'sequence': 'movie release is leaked by a tamil ',
  'token': 263,
  'token_str': ' a'},
 {'score': 0.02664225921034813,
  'sequence': 'movie release is leaked by any tamil ',
  'token': 495,
  'token_str': ' any'},
 {'score': 0.018089940771460533,
  'sequence': 'movie release is leaked by its tamil ',
  'token': 415,
  'token_str': ' its'}]